__1. Парсинг всех матчей, нужно поменять значение `max_pages`(если на сайте изменилось кол-во страниц), сейчас их 28 и вставить актуальный путь к браузеру.__

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time

chrome_driver_path = '/Users/vovidze/Downloads/chromedriver-mac-arm64 2/chromedriver'

chrome_service = Service(executable_path=chrome_driver_path)

offset_step = 100
max_pages = 27
team_links = []

for page_number in range(max_pages):
    driver_links = webdriver.Chrome(service=chrome_service)

    url = "https://www.hltv.org/results?stars=2&matchType=Online"

    current_url = f"{url}&offset={page_number * offset_step}"

    driver_links.get(current_url)

    wait = WebDriverWait(driver_links, 1)
    popup_locator = (By.CLASS_NAME, "CybotCookiebotDialogContentWrapper")
    cookie_popup = wait.until(EC.presence_of_element_located(popup_locator))

    cookie_button_locator = (By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")
    cookie_button = cookie_popup.find_element(*cookie_button_locator)
    cookie_button.click()

    team_links_parse = True
    if team_links_parse:
        try:
            new_xpath = "result-con"
            team_elements = driver_links.find_elements(By.CLASS_NAME, new_xpath)

            for team_element in team_elements:
                href_element = team_element.find_element(By.TAG_NAME, 'a')
                href = href_element.get_attribute("href")
                team_links.append(href)
        except Exception as e:
            print("Error occurred while fetching team links.")
    driver_links.quit()
        

print(team_links)

with open('team_links.txt', 'w') as file:
    for link in team_links:
        file.write(link + '\n')

['https://www.hltv.org/matches/2369282/mouz-vs-spirit-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-b', 'https://www.hltv.org/matches/2369273/eternal-fire-vs-mouz-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-b', 'https://www.hltv.org/matches/2369248/natus-vincere-vs-virtuspro-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-a', 'https://www.hltv.org/matches/2369249/cloud9-vs-betboom-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-a', 'https://www.hltv.org/matches/2369239/big-vs-betboom-pgl-cs2-major-copenhagen-2024-europe-rmr-closed-qualifier-a', 'https://www.hltv.org/matches/2368226/movistar-riders-vs-apeks-esl-challenger-atlanta-2023-europe-closed-qualifier', 'https://www.hltv.org/matches/2367984/eternal-fire-vs-big-betboom-dacha-2023-closed-qualifier', 'https://www.hltv.org/matches/2367980/apeks-vs-big-betboom-dacha-2023-closed-qualifier', 'https://www.hltv.org/matches/2367998/apeks-vs-spirit-esl-challenger-league-season-46-europe', 'https

__2. Функция для парсинга каждого матча__

In [3]:
with open('team_links_online.txt', 'r') as file:
    team_links = [line.strip() for line in file]

In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import os

intermediate_file_path = "intermediate_results.xlsx"

def fetch_player_data(match_urls):
    global processed_urls
    processed_urls = 0
    
    chrome_driver_path = '/Users/vovidze/Downloads/chromedriver-mac-arm64/chromedriver'
    chrome_service = Service(executable_path=chrome_driver_path)
    chrome_options = webdriver.ChromeOptions()
    userAgent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.56 Safari/537.36"
    chrome_options.add_argument(f'user-agent={userAgent}')
    chrome_options.add_argument('--headless')

    players_data_all_matches = []

    for match_url in match_urls:
        driver_match = webdriver.Chrome(service=chrome_service, options=chrome_options)
        driver_match.get(match_url)

        wait = WebDriverWait(driver_match, 1)

        popup_locator = (By.CLASS_NAME, "CybotCookiebotDialogContentWrapper")
        cookie_popup = wait.until(EC.presence_of_element_located(popup_locator))

        cookie_button_locator = (By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")
        cookie_button = cookie_popup.find_element(*cookie_button_locator)
        cookie_button.click()

        tournament_locator = (By.XPATH, '//*[@class="event text-ellipsis"]/a')
        tournament_name_element = driver_match.find_element(*tournament_locator)
        tournament_name = tournament_name_element.text
        tournament_url = tournament_name_element.get_attribute("href")

        team1_locator = (By.XPATH, '//*[@class="team1-gradient"]/div') 
        team1_element = driver_match.find_element(*team1_locator)
        team1_score = team1_element.text

        team2_locator = (By.XPATH, '//*[@class="team2-gradient"]/div') 
        team2_element = driver_match.find_element(*team2_locator)
        team2_score = team2_element.text

        team1_url_locator = (By.XPATH, '//*[@class="team1-gradient"]/a') 
        team1_url_element = driver_match.find_element(*team1_url_locator)
        team1_url = team1_url_element.get_attribute("href")

        team2_url_locator = (By.XPATH, '//*[@class="team2-gradient"]/a') 
        team2_url_element = driver_match.find_element(*team2_url_locator)
        team2_url = team2_url_element.get_attribute("href")
        
        team1_country_locator = (By.XPATH, '//*[@class="team1"]') 
        team1_country_element = driver_match.find_element(*team1_country_locator)
        team1_country = team1_country_element.get_attribute("title")

        team2_country_locator = (By.XPATH, '//*[@class="team2"]') 
        team2_country_element = driver_match.find_element(*team2_country_locator)
        team2_country = team2_country_element.get_attribute("title")

        time_locator = (By.CSS_SELECTOR, 'body > div.bgPadding > div.widthControl > div:nth-child(2) > div.contentCol > div.match-page > div.standard-box.teamsBox > div.timeAndEvent > div.time')
        time_element = driver_match.find_element(*time_locator)
        time = time_element.text

        date_locator = (By.CSS_SELECTOR, 'body > div.bgPadding > div.widthControl > div:nth-child(2) > div.contentCol > div.match-page > div.standard-box.teamsBox > div.timeAndEvent > div.date')
        date_element = driver_match.find_element(*date_locator)
        date1 = int(date_element.get_attribute("data-unix"))/1000
        date = datetime.utcfromtimestamp(date1).date()
        formatted_date = date.strftime("%Y-%m-%d")
        

        players = []
        try:
            player_elements = driver_match.find_elements(By.XPATH, "//td[@class='players']//div[@class='flagAlign']/a")

            for player_element in player_elements:
                href = player_element.get_attribute("href")
                player_name_element = player_element.find_element(By.CLASS_NAME, 'player-nick')
                player_name = player_name_element.text
                players.append(href)
        except Exception as e:
            print(f"Error occurred while fetching players: {e}")

        players1 = []
        count = 0
        for i in players: 
            if i not in players1: 
                count += 1
                players1.append(i) 

        players_data_table1 = []
        try:
            table1_locator = (By.XPATH, '//*[@id="all-content"]/table[1]')
            table1 = wait.until(EC.presence_of_element_located(table1_locator))

            rows_table1 = table1.find_elements(By.TAG_NAME, 'tr')[1:] # Пропускаем заголовок таблицы
            team1_name = table1.find_elements(By.TAG_NAME, 'tr')[0].text.split("\n")[0]
            for row, num in zip(rows_table1, range(0, 5)):
                columns = row.find_elements(By.TAG_NAME, 'td')
                player_name = columns[0].text
                kd = columns[1].text
                plus_minus = columns[2].find_element(By.TAG_NAME, 'span').text
                adr = columns[3].text
                kast = columns[4].text
                rating = columns[5].text
                link = players1[num]
                country = columns[0].find_element(By.TAG_NAME, 'img').get_attribute("title")
                try:
                    photo_elements_locator = (By.CLASS_NAME, "player-photo")
                    photo_elements = wait.until(EC.presence_of_all_elements_located(photo_elements_locator))

                    for photo_element in photo_elements:
                        player_nickname_in_alt = photo_element.get_attribute("alt")

                        if player_name.lower() in player_nickname_in_alt.lower():
                            player_photo_url = photo_element.get_attribute("src")

                except Exception as e:
                    print(f"Error occurred while fetching player photo: {e}")

                photo_link = player_photo_url

                player_data = {
                    'id': link.split('/')[4],
                    'Player Name': player_name,
                    'K/D': kd,
                    'Plus/Minus': plus_minus,
                    'ADR': adr,
                    'KAST': kast,
                    'Rating': rating,
                    'Country': country,
                    'Team country': team1_country,
                    'Player link': link,
                    'Photo link': photo_link,
                    'Match link': match_url,
                    'Team link': team1_url,
                    'Tournament link': tournament_url,
                    'Team Name': team1_name,
                    'Team id': team1_url.split('/')[4],
                    'Tournament Name': tournament_name, 
                    'Date': formatted_date,
                    'Time': time,
                    'Allies score': team1_score,
                    'Opponents score': team2_score,
                    'Winner': 1 if team1_score > team2_score else 0,
                    'Tournament type': "Online"
                }

                players_data_table1.append(player_data)

        except Exception as e:
            print(f"Error occurred while fetching player data from table 1: {e} {match_url}")
            
        players_data_table4 = []
        try:
            table4_locator = (By.XPATH, '//*[@id="all-content"]/table[4]')
            table4 = wait.until(EC.presence_of_element_located(table4_locator))

            rows_table4 = table4.find_elements(By.TAG_NAME, 'tr')[1:]
            team2_name = table4.find_elements(By.TAG_NAME, 'tr')[0].text.split("\n")[0]
            for row, num in zip(rows_table4, range(5, 10)):
                columns = row.find_elements(By.TAG_NAME, 'td')
                player_name = columns[0].text
                kd = columns[1].text
                plus_minus = columns[2].find_element(By.TAG_NAME, 'span').text
                adr = columns[3].text
                kast = columns[4].text
                rating = columns[5].text
                link = players1[num]
                country = columns[0].find_element(By.TAG_NAME, 'img').get_attribute("title")
                try:
                    photo_elements_locator = (By.CLASS_NAME, "player-photo")
                    photo_elements = wait.until(EC.presence_of_all_elements_located(photo_elements_locator))

                    for photo_element in photo_elements:
                        player_nickname_in_alt = photo_element.get_attribute("alt")

                        if player_name.lower() in player_nickname_in_alt.lower():
                            player_photo_url = photo_element.get_attribute("src")

                except Exception as e:
                    print(f"Error occurred while fetching player photo: {e}")

                photo_link = player_photo_url

                player_data = {
                    'id': link.split('/')[4],
                    'Player Name': player_name,
                    'K/D': kd,
                    'Plus/Minus': plus_minus,
                    'ADR': adr,
                    'KAST': kast,
                    'Rating': rating,
                    'Country': country,
                    'Team country': team2_country,
                    'Player link': link,
                    'Photo link': photo_link,
                    'Match link': match_url,
                    'Team link': team2_url,
                    'Tournament link': tournament_url,
                    'Team Name': team2_name,
                    'Team id': team2_url.split('/')[4],
                    'Tournament Name': tournament_name,
                    'Date': formatted_date,
                    'Time': time,
                    'Allies score': team2_score,
                    'Opponents score': team1_score,
                    'Winner': 1 if team2_score > team1_score else 0,
                    'Tournament type': "Online"
                }

                players_data_table4.append(player_data)

            driver_match.quit()

        except Exception as e:
            print(f"Error occurred while fetching player data from table 4: {e}")

        players_data_all_matches.extend(players_data_table1)
        players_data_all_matches.extend(players_data_table4)
        
        processed_urls += 1
        
        if processed_urls % 10 == 0: # промежуточное сохранение в файл каждые 10 итераций
            if os.path.exists(intermediate_file_path):
                with pd.ExcelWriter(intermediate_file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
                    df_intermediate = pd.DataFrame(players_data_all_matches)
                    df_intermediate.to_excel(writer, index=False, header=False)
            else:
                df_intermediate = pd.DataFrame(players_data_all_matches)
                df_intermediate.to_excel(intermediate_file_path, index=False)
        if processed_urls % 10 == 0 and processed_urls > 0:
            print(f"{processed_urls} ссылок успешно спаршены.")

        driver_match.quit()

    df_all_matches = pd.DataFrame(players_data_all_matches)
    df_all_matches.to_excel("final_results.xlsx", index=False)
    
    return df_all_matches

In [14]:
import time

match_urls = team_links[1290:]

result_df = fetch_player_data(match_urls)

print(f"Execution time: {execution_time} seconds")
result_df

10 ссылок успешно спаршены.
20 ссылок успешно спаршены.
30 ссылок успешно спаршены.
40 ссылок успешно спаршены.
50 ссылок успешно спаршены.
60 ссылок успешно спаршены.
70 ссылок успешно спаршены.
80 ссылок успешно спаршены.
90 ссылок успешно спаршены.
100 ссылок успешно спаршены.
110 ссылок успешно спаршены.
Error occurred while fetching player data from table 1: Message: 
Stacktrace:
0   chromedriver                        0x000000010454a814 chromedriver + 4040724
1   chromedriver                        0x0000000104542a18 chromedriver + 4008472
2   chromedriver                        0x00000001041b5870 chromedriver + 284784
3   chromedriver                        0x00000001041f9080 chromedriver + 561280
4   chromedriver                        0x0000000104233048 chromedriver + 798792
5   chromedriver                        0x00000001041ed74c chromedriver + 513868
6   chromedriver                        0x00000001041ee044 chromedriver + 516164
7   chromedriver                        0x0

KeyboardInterrupt: 

In [99]:
result_df.to_excel('output.xlsx', index=False)

In [51]:
team_links.index("https://www.hltv.org/matches/2336874/cr4zy-vs-tricked-esea-mdl-season-32-europe")

959

__3. Скачиваем все изображения игроков__

In [129]:
import os
import httplib2
from PIL import Image
from io import BytesIO

output_folder = "/Users/vovidze/Desktop/fwhrpaper/images"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
df_all_matches = result_df

http = httplib2.Http()

# Итерация по датасету для загрузки фотографий
for index, row in df_all_matches.iterrows():
    photo_link = row['Photo link']
    player_id = row['id']

    if photo_link:  # Проверяем, что ссылка на фото существует
        # Загружаем содержимое фотографии
        response, content = http.request(photo_link)

        # Создаем изображение из полученного содержимого
        image = Image.open(BytesIO(content))

        # Формируем имя файла для сохранения с использованием ID игрока
        file_name = f"{player_id}.jpg"

        # Путь для сохранения изображения
        image_path = os.path.join(output_folder, file_name)

        # Сохраняем изображение
        image.save(image_path, 'JPEG')

print("Изображения успешно скачаны и сохранены с именами, соответствующими ID игроков.")

Изображения успешно скачаны и сохранены с именами, соответствующими ID игроков.
